In [1]:
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch, cm

from copy import copy

In [19]:
keywords = {
    '\t': '  ',
#     'stubborness': '[#]',
#     'perception': '(O)',
    'defense': 'AC',
#     'on block,': 'block ='
    'stubborness': 'toughness'
}

def process(line):
    line = line[1:]
    for word, symbol in keywords.items():
        line = line.replace(word, symbol)
    return line


cards = {}
class Card:
    def __init__(self, lines):
#         lines = raw.strip().split('\n')
        self.title = lines[0]
        cards[self.title] = self
        self.lines = [process(line) for line in lines[1:]]
        pack = None
        
    def render(self, c, xoffset, yoffset):

        xoffset += .45*cm
        yoffset -= .25*cm
        
        def write(text='', fontsize=14):
            nonlocal yoffset
            
            if text.startswith(keywords['\t']):
                yoffset -= 2
            else:
                yoffset -= 7
            
            c.setFont('Helvetica', fontsize)      
            yoffset -= fontsize
            c.drawString(xoffset, yoffset, text)

        write(self.title, 18)
        
        yoffset -= .12*cm
        xoffset += .2*cm
        for line in self.lines:
            write(line)

            
# read in all cards

def parse(fpath, cls):
    buffer = []
    for line in open(fpath):
        if line.startswith('--'):
            continue
            
        buffer.append(line[:-1])
        if line.strip() == '' and buffer:
            cls(buffer[:-1])
            buffer = []

parse('../card_data', Card)
print(len(cards), 'unique cards')

packs = {}
class Pack:
    def __init__(self, lines):
        self.title = lines[0]
        packs[self.title] = self
        
        self.cards = []
        
        titlecard = Card(lines)
        titlecard.lines.insert(0, '---- pack ----')
        self.cards.append(titlecard)
        
        for line in lines[1:]:
            words = line.strip().split()
            count = int(words[0][0])
            name = ' '.join(words[1:])
            
            if name not in cards:
                print(f'unknown card <{name}>')
            else:
                card = copy(cards[name])
                card.pack = ''.join(word[0] for word in self.title.split())
                self.cards += count*[cards[name]]
        
#         print(self.cards[0].lines)
#         print(self.cards[1].lines)
#         input()
    
parse('../pack_data', Pack)
print(len(packs), 'unique packs')

width = 8*inch
height = 11*inch

rows = 4
cols = 3

class Document:    
    def __init__(self, name=None):
        self.c = canvas.Canvas(name)
        self.c.setPageSize((width, height))
        
        self.row = 0
        self.col = 0

    def nextPage(self):
        self.c.setStrokeColorRGB(.5,.5,.5)
        
        for col in range(cols):
            self.col = col+1            
            self.c.line(self.xoffset, 0, self.xoffset, height)
            
        for row in range(rows):
            self.row = row+1
            self.c.line(0, self.yoffset, width, self.yoffset)
            
        self.c.showPage()
    
    def proceed(self):
        self.col += 1
        if self.col == cols:
            self.col = 0
            self.row += 1
            if self.row == rows:
                self.nextPage()
                self.row = 0
                self.col = 0
    
    @property
    def xoffset(self):
        return width * (self.col/cols)
    
    @property
    def yoffset(self):
        return height*(1- self.row/rows)
    
    def render(self, card):
        card.render(self.c, self.xoffset, self.yoffset)
        self.proceed()

        
doc = Document('out.pdf')

to_show = (
    packs['Elven Spear'], packs['Core Cleric Track'], 
    packs['Core Fighter Track'], packs['Core Rogue Track']
)

for pack in to_show[::-1]:
    for card in pack.cards:
        doc.render(card)

doc.nextPage()
doc.c.save()

'done'

42 unique cards
15 unique packs


'done'

In [16]:
packs

{"Bandit's Club": <__main__.Pack at 0x10e22aeb8>,
 'Core Cleric Track': <__main__.Pack at 0x10e22acf8>,
 'Core Fighter Track': <__main__.Pack at 0x10e22ac50>,
 'Core Rogue Track': <__main__.Pack at 0x10e22ae48>,
 'Disciple of Air': <__main__.Pack at 0x10e2482b0>,
 'Disciple of Earth': <__main__.Pack at 0x10e2481d0>,
 'Elven Spear': <__main__.Pack at 0x10e248128>,
 'Goblin Scimitar': <__main__.Pack at 0x10e22add8>,
 'Hunting Bow': <__main__.Pack at 0x10e22aef0>,
 'Icy Warhammer': <__main__.Pack at 0x10e248240>,
 "Recruit's Chainmail": <__main__.Pack at 0x10e248048>,
 'Shortsword': <__main__.Pack at 0x10e22ad30>,
 "Skirmisher's Shield": <__main__.Pack at 0x10e22ae80>,
 "Thief's Dagger": <__main__.Pack at 0x10e22afd0>,
 'Tower Shield': <__main__.Pack at 0x10e2480b8>}